In [1]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [16]:
df_steam_games = pq.read_table('df_steam_games_1.parquet').to_pandas()

In [17]:
# Selección de columnas relevantes
data = df_steam_games[['id', 'app_name', 'genres', 'tags', 'specs']]

In [18]:
# Limpieza y preprocesamiento de texto
data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']

C:\Users\Blasferp\AppData\Local\Temp\ipykernel_6792\2149226059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_features'] = data['genres'] + ' ' + data['tags'] + ' ' + data['specs']


In [19]:
data.shape

(2020050, 6)

In [20]:
data = data.groupby('id').first().reset_index()

In [21]:
data = data[data['id']!=0]

In [22]:
data['app_name'].nunique()

30708

In [23]:
data = data.sample(n=3500)

In [24]:
data.shape

(3500, 6)

In [25]:
data.reset_index(drop=True, inplace=True)

In [26]:
data

,id,app_name,genres,tags,specs,combined_features
0,708190,code world,indie,indie,single player,indie indie single player
1,548050,runes,casual,casual,single player,casual casual single player
2,497930,wanda a graphic novel,casual,rpg,single player,casual rpg single player
3,632740,equilibrium of divinity original soundtrack,indie,rpg,single player,indie rpg single player
4,367220,egyptian settlement gold,casual,strategy,single player,casual strategy single player
...,...,...,...,...,...,...
3495,301245,fantasy grounds 4e: lands of darkness #2: ce...,indie,rpg,multi player,indie rpg multi player
3496,775210,jump gunners original soundtrack,action,action,single player,action action single player
3497,689600,little earth,none,early access,single player,none early access single player
3498,494960,twisted arrow,none,action,single player,none action single player


In [27]:
data.head(10)

,id,app_name,genres,tags,specs,combined_features
0,708190,code world,indie,indie,single player,indie indie single player
1,548050,runes,casual,casual,single player,casual casual single player
2,497930,wanda a graphic novel,casual,rpg,single player,casual rpg single player
3,632740,equilibrium of divinity original soundtrack,indie,rpg,single player,indie rpg single player
4,367220,egyptian settlement gold,casual,strategy,single player,casual strategy single player
5,211800,dead hungry diner,casual,casual,single player,casual casual single player
6,407821,pool nation fx unlock balls,casual,strategy,single player,casual strategy single player
7,342650,fermis path,action,indie,single player,action indie single player
8,520930,pong it vr,none,early access,single player,none early access single player
9,237621,batman: arkham origins infinite earths skin ...,action,action,single player,action action single player


In [28]:
data['app_name'].value_counts()

app_name
total extreme wrestling                                               2
fortify                                                               2
toukiden kiwami   mission collection 1 3                              1
aliens vs. predator™                                                  1
rocksmith® 2014 – my chemical romance   “i’m not okay (i promise)”    1
                                                                     ..
创世 修真录                                                                1
timore 5                                                              1
slayaway camp                                                         1
fourchords guitar karaoke   meghan trainor song pack                  1
witches heroes and magic                                              1
Name: count, Length: 3498, dtype: int64

In [29]:
import pyarrow.parquet as pq
data.to_parquet('df_recomendacion.parquet')

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(data['combined_features'])

In [32]:
cosine_similarity = cosine_similarity(converted_metrix)

In [33]:
def get_recommendations(app_name, cosine_sim=cosine_similarity ):
    idx = data[data['app_name'] == app_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 10 juegos similares
    game_indices = [i[0] for i in sim_scores]
    return data['app_name'].iloc[game_indices]



In [34]:
# guardo la matriz

import joblib #pickle es alternativa

joblib.dump(cosine_similarity, 'Matriz.pkl', compress=1)

['Matriz.pkl']

In [35]:
import joblib
with open('Matriz.pkl', 'rb') as file:
        modelo = joblib.load(file)

data = pd.read_parquet('df_recomendacion.parquet')

In [36]:
data.shape

(3500, 6)

In [37]:
recommendations = get_recommendations('fortify')
print(recommendations)

128    toy soldiers: war chest   legendary heroes
233                             executive assault
259                            his   extend  pack
311                    mechs  mercs: black talons
339         bloons td 5   top gun monkey ace skin
Name: app_name, dtype: object
